# SQL Data Query Language Queries - One Page

- A one page reference for common Data Query Language Commands, all in one place.
- Extensive notes are added to more difficult concepts such as correlated subqueries, cube, rollup etc. Notes aren't added to every example.  
- Some notes and examples are my own, some notes and examples are from online tutorials.
- To keep the length down, table creation is not included BUT the tables are usually logical in describing VARCHAR, INTS, BOOLEANS etc.

- If you just want to review the syntax of these queries visit: 
https://github.com/richardgourley/sql-one-pagers/blob/main/sql-dql-syntax-only.ipynb

**CONTENTS**

**1. SQL Query Order**

**2. Sorting**

**3. Limiting**

**4. Filtering**

**5. Joining Tables**

**6. Grouping**

**7. SubQuery**

**8. Set Operators**

**9. Common Table Expressions**

**10. Pivot**

**11. Expressions - Coalesce, Case, NullIf**

# 1. SQL Query Order

**SQL queries are evaluated in the following order:**

**FROM** - choose and join tables

**WHERE** - filters the data

**GROUP BY** - aggregates the data

**HAVING** - filters the aggregated data

**SELECT** - returns the final data

**ORDER BY** - sorts the final data

**LIMIT** - limits sorted data to a row count

-----

# 2. SORTING

## 2a. Order By

```SQL
SELECT 
    first_name
FROM
    sales.customers
ORDER BY 
    first_name DESC
```

--

```SQL
SELECT 
    first_name
FROM
    sales.customers
ORDER BY 
    city,
    first_name
```

--

```SQL
SELECT
    first_name
FROM
    sales.customers
ORDER BY 
    LEN(first_name) DESC
```

-----

# 3. LIMITING

## 3a. Offset, Fetch

```SQL
SELECT 
    team_name,
    points,
FROM 
    prem_table
ORDER BY 
    points DESC
OFFSET 4 ROWS
FETCH FIRST 13 ROWS
```

## 3b. Select Top

```SQL
SELECT TOP 10
SELECT TOP 1 PERCENT
SELECT TOP 3 WITH TIES
```

----

# 4. FILTERING

## 4a. Distinct

```SQL
SELECT DISTINCT 
    country
FROM 
    sales.customers
```

## 4b. And, Or, In

```SQL
SELECT 
    product_name,
    brand_id,
    list_price
FROM 
    production.products
WHERE
    (brand_id 1 OR brand_id = 2)
AND
    list_price > 40
```

--

```SQL
SELECT 
    product_id,
    brand_id
FROM
    products
WHERE
    brand_id IN(1,2,3,4)
```

## 4c. Between, Not Between

```SQL
SELECT
    product_id,
    product_name,
    list_price
FROM
    production.products
WHERE
    list_price NOT BETWEEN 149.99 AND 199.99
```

--

```SQL
SELECT 
    order_id,
    order_date
FROM
    orders
WHERE
    order_date BETWEEN '20170115' AND '20170117
```

## 4d. Like

```SQL
WHERE
    last_name LIKE 'z%'
```

```SQL
WHERE
    last_name LIKE '%er'
```

```SQL
WHERE
    last_name LIKE 't%s'
```

```SQL
WHERE
    last_name LIKE '_u%'
```

```SQL
WHERE
    last_name LIKE '[ZY]%'
```

```SQL
WHERE
    last_name LIKE '[^A-X]%'
```

```SQL
WHERE 
    comment LIKE '%30!%%' ESCAPE '!'
```

## 4e. Column Aliases

```SQL
SELECT
    first_name + ' ' + last_name AS full_name
```

--

```SQL
SELECT
    first_name + ' ' + last_name AS 'Full Name'
```


--

```SQL
SELECT
    c.customer_id,
    first_name,
FROM
    sales.customers c
INNER JOIN sales.orders o ON o.customer_id = c.customer_id
```

------

# 5. Joining Tables

## 5a. Inner Join

```SQL
SELECT 
    c.id candidate_id,
    c.full_name candidate_name,
    e.id employeed_id,
    e.full_name employee_name
FROM 
    candidates c
    INNER JOIN 
        employees e
        ON c.id = e.id
```

--

```SQL
SELECT 
    product_name, 
    category_name,
    list_price
FROM 
    production.products p
    INNER JOIN production.categories c
        ON p.category_id = c.category_id
```

## 5b. Left Join

```SQL
SELECT  
    c.id candidate_id,
    c.fullname candidate_name,
    e.id employee_id,
    e.fullname employee_name
FROM 
    candidates c
    LEFT JOIN employees e 
        ON e.fullname = c.fullname
```

--

```SQL
SELECT
    product_name,
    order_id
FROM
    production.products p
    LEFT JOIN sales.order_items o 
        ON o.product_id = p.product_id
    ORDER BY
        order_id;
```

## 5c. Right Join

```SQL
SELECT 
    c.id candidate_id,
    c.full_name candidate_name,
    e.id employee_id,
    e.full_name employee_name
FROM 
    candidates c
    RIGHT JOIN employees e
        ON c.full_name = e.full_name
```

## 5d. Full Outer Join

```SQL
SELECT  
    c.id candidate_id,
    c.fullname candidate_name,
    e.id employee_id,
    e.fullname employee_name
FROM 
    hr.candidates c
    FULL JOIN hr.employees e 
        ON e.fullname = c.fullname;
```

## 5e. Cross Join

```SQL
SELECT
    s.store_id,
    p.product_id,
    ISNULL(sales, 0) sales
FROM
    sales.stores s
CROSS JOIN production.products p
    LEFT JOIN (....SUB QUERY...)

```

## 5f. Self Join

```SQL
SELECT
    e.full_name employee,
    m.full_name manager
FROM 
    employees e
    LEFT JOIN employees m
    ON m.employee_id = e.manager_id
```

-----

# 6. Grouping

## 6a. Group By

```SQL
SELECT
    customer_id,
    YEAR (order_date) order_year,
    COUNT (order_id) orders_placed
FROM
    sales.orders
WHERE
    customer_id IN (1, 2)
GROUP BY
    customer_id,
    YEAR (order_date)
ORDER BY
    customer_id

```

--

```SQL
SELECT
    city,
    COUNT (customer_id) customer_count
FROM
    sales.customers
GROUP BY
    city
ORDER BY
    city
```

--

```SQL
SELECT
    brand_name,
    AVG (list_price) avg_price
FROM
    production.products p
INNER JOIN production.brands b ON b.brand_id = p.brand_id
WHERE
    model_year = 2018
GROUP BY
    brand_name
ORDER BY
    brand_name
```

--

```SQL
SELECT
    order_id,
    SUM (
        quantity * list_price * (1 - discount)
    ) net_value
FROM
    sales.order_items
GROUP BY
    order_id;

```

## 6b. Having

```SQL
SELECT 
    salesperson_name,
    SUM(sales) sales_total
FROM 
    sales
GROUP BY 
    salesperson_name
HAVING 
    SUM(sales) < 2000
```

--

```SQL
SELECT
    category_id,
    MAX (list_price) max_list_price,
    MIN (list_price) min_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    MAX (list_price) > 4000 OR MIN (list_price) < 500
```

--

```SQL
SELECT
    category_id,
    AVG (list_price) avg_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    AVG (list_price) BETWEEN 500 AND 1000
```

## 6c. Grouping Sets

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    GROUPING SETS (
        (brand, category),
        (brand),
        (category),
        ()
    )
ORDER BY
    brand,
    category
```

## 6d. Cube

```SQL
GROUPING SETS (
        (d1,d2,d3), 
        (d1,d2),
        (d1,d3),
        (d2,d3),
        (d1),
        (d2),
        (d3), 
        ()
     )
```

--

```SQL
SELECT
    d1,
    d2,
    d3,
    aggregate_function (c4)
FROM
    table_name
GROUP BY
    CUBE (d1, d2, d3)
```

--

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    brand,
    CUBE(category)
```

## 6e. Roll Up

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    ROLLUP(brand, category)
```

--

```SQL
SELECT
    category,
    brand,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    ROLLUP (category, brand)
```

--

```SQL
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    brand,
    ROLLUP (category)
```

-----

# 7. SubQuery

## 7a. Overview

**QUERY DESCRIPTION:**
- This query returns all orders where the customer is based in New York.
- The sub query finds all customers in New York.
- The outer query then gets order details from the order table but only if the customer_id appears in the 'New York' customers sub table.

```SQL
SELECT
    order_id,
    order_date,
    customer_id
FROM
    sales.orders
WHERE
    customer_id IN (
        SELECT
            customer_id
        FROM
            sales.customers
        WHERE
            city = 'New York'
    )
ORDER BY
    order_date DESC
```

--

**QUERY AIM:**
- This query finds the highest list price item from ORDER ITEMS for every single order from the ORDERS table.

**NOTES:**
- The subquery takes the order_id from the outer query, finds all order_items for that order_id and returns the MAX list_price from the order.

```SQL 
SELECT
    order_id,
    order_date,
    (
        SELECT
            MAX (list_price)
        FROM
            sales.order_items i
        WHERE
            i.order_id = o.order_id
    ) AS max_list_price
FROM
    sales.orders o
ORDER BY order_date desc
```

## 7b. Correlated SubQuery

**QUERY AIM:**
- This query finds all products that have the MAX list_price in it's category.

**NOTES:**
- Correlated subqueries rely on the outer table.
- Typicallly, a column from the outer table is matched to the table in the subquery.
- The table in the subquery can be the same table as the outer table or a different table.

```SQL
SELECT
    product_name,
    list_price,
    category_id
FROM
    production.products p1
WHERE
    list_price IN (
        SELECT
            MAX (p2.list_price)
        FROM
            production.products p2
        WHERE
            p2.category_id = p1.category_id
        GROUP BY
            p2.category_id
    )
ORDER BY
    category_id,
    product_name
```

## 7c. Exists

**NOTES:**
- An EXISTS query is a correlated subquery in that it relies on a match from something in the outer query.

**QUERY DESCRIPTION:**
- This query returns all customers who have placed more than 2 orders in the orders table.
- For every customer_id, the subquery counts up the number of orders placed for that customer_id, returning True if more than 2, or False if less.

```SQL
SELECT
    customer_id,
    first_name,
    last_name
FROM
    sales.customers c
WHERE
    EXISTS (
        SELECT
            COUNT (*)
        FROM
            sales.orders o
        WHERE
            customer_id = c.customer_id
        GROUP BY
            customer_id
        HAVING
            COUNT (*) > 2
    )
ORDER BY
    first_name,
    last_name
```

--

**NOTES:**
- Sometimes a simpler subquery using IN can achieve the same results, without the need for a correlated subquery.
- This query example is a simpler way to find all orders where the customer is based in San Jose.

```SQL
SELECT
    *
FROM
    sales.orders
WHERE
    customer_id IN (
        SELECT
            customer_id
        FROM
            sales.customers
        WHERE
            city = 'San Jose'
    )
ORDER BY
    customer_id,
    order_date
```

--

**QUERY DESCRIPTION:**
- This query gets orders where the customer is based in San Jose using a correlated subquery and EXISTS.
- The sub query takes the customer_id for each order and finds matches from the customer table BUT only if the city of that customer is San Jose, returning True or False.

```SQL
SELECT
    *
FROM
    sales.orders o
WHERE
    EXISTS (
        SELECT
            customer_id
        FROM
            sales.customers c
        WHERE
            o.customer_id = c.customer_id
        AND city = 'San Jose'
    )
ORDER BY
    o.customer_id,
    order_date
```

## 7d. Any

**QUERY DESCRIPTION:**
- This query gets all products who have appeared 2 or more times as an order item.
- For each item in the outer query, the subquery is run and if the product_id is found to match ANY of the product_ids returned by the subquery, then the result is true and the product is returned.

```SQL
SELECT
    product_name,
    list_price
FROM
    production.products
WHERE
    product_id = ANY (
        SELECT
            product_id
        FROM
            sales.order_items
        WHERE
            quantity >= 2
    )
ORDER BY
    product_name
```

## 7e. All

**QUERY DESCRIPTION:**
- This query finds all products that have a higher list_price than ALL of the AVERAGE list_price for every brand_id.
- The subquery returns the AVERAGE list_price of every brand_id.
- Every product is assessed and the list_price of the product must be higher than the AVERAGE of ALL brands.

```SQL
SELECT
    product_name,
    list_price
FROM
    production.products
WHERE
    list_price > ALL (
        SELECT
            AVG (list_price) avg_list_price
        FROM
            production.products
        GROUP BY
            brand_id
    )
ORDER BY
    list_price
```

-----

# 8. Set Operators

## 8a. Union

```SQL
SELECT
    first_name,
    last_name
FROM
    sales.staffs
UNION
SELECT
    first_name,
    last_name
FROM
    sales.customers
```

## 8b. Union All

```SQL
SELECT
    first_name,
    last_name
FROM
    sales.staffs
UNION ALL
SELECT
    first_name,
    last_name
FROM
    sales.customer
```

## 8c. Intersect

```SQL
SELECT 
    player_name
FROM 
    player_of_the_year 2021
INTERSECT
SELECT 
    player_id
FROM 
    player_of_the_year_2020
```

## 8d. Except

```SQL
SELECT 
    first_name,
    last_name
FROM
    employees
EXCEPT
SELECT
    first_name,
    last_name
FROM 
    completed_health_and_safety
```

-----

# 9. Common Table Expressions

## 9a. CTEs

```SQL
WITH sales_person_yearly_total (sales_person_id, sales, year) AS (
SELECT
    id,
    SUM(sales_total)
    YEAR(sales_date)
FROM 
    sales
GROUP BY 
    id,
    YEAR(sales_date)
)
SELECT 
    sales_person_id,
    sales,
    year
FROM 
    sales_person_yearly_total
WHERE
    year = 2019

```

--

```SQL
WITH cte_sales AS (
    SELECT 
        staff_id, 
        COUNT(*) order_count  
    FROM
        sales.orders
    WHERE 
        YEAR(order_date) = 2018
    GROUP BY
        staff_id
)
SELECT
    AVG(order_count) average_orders_by_staff
FROM 
    cte_sales
```

--

```SQL
WITH cte_1 AS (
SELECT ....
),
WITH cte_2 AS (
SELECT ...
)
SELECT 
    col_a,
    col_b
FROM 
    cte_1
    INNER JOIN cte_2 ON...
WHERE...
ORDER BY...

```

## 9b. Recursive CTEs

```SQL
WITH cte ( value )
  AS (
       SELECT   1
       UNION ALL
       SELECT   value + 1
       FROM     cte
       WHERE    value < 4
     )
SELECT  *
FROM    cte
```

--

```SQL
WITH organization AS
(
SELECT 
    employee_id,
    full_name,
    manager_id
FROM
    employees
WHERE
    manager_id IS NULL
UNION ALL
SELECT 
    employees.employee_id,
    employees.full_name, 
    employees.manager_id
FROM
    employees
    INNER JOIN organization
    ON organization.employee_id = employees.manager_id
)
SELECT *
FROM 
organization
```

--

```SQL
WITH outer_products AS
(
SELECT 
    item_id,
    item_name,
    product_id
FROM
    products
WHERE
    product_id IS NULL
UNION ALL
SELECT 
    p.item_id,
    p.item_name, 
    p.product_id
FROM
    products p
    INNER JOIN outer_products o
    ON o.item_id = p.product_id
)
SELECT *
FROM 
outer_products
```

-----

# 10. Pivot

## 10a. Pivot

```SQL
SELECT * FROM   
(
    SELECT 
        category_name, 
        product_id
    FROM 
        products p
        INNER JOIN categories c 
            ON c.category_id = p.category_id
) t 
PIVOT(
    COUNT(product_id) 
    FOR category_name IN (
        [CatA], 
        [CatB], 
        [CatC], 
        [CatD]
) AS pivot_table
```

--

```SQL
SELECT * FROM   
(
    SELECT 
        category_name, 
        product_id,
        model_year
    FROM 
        products p
        INNER JOIN categories c 
            ON c.category_id = p.category_id
) t 
PIVOT(
    COUNT(product_id) 
    FOR category_name IN (
        [CatA], 
        [CatB], 
        [CatC], 
        [CatD]
) AS pivot_table
```

-----

# 11. Expressions

## 11a. Case

```SQL
SELECT 
    SUM(CASE
        WHEN test_score > 70
        THEN 1
        ELSE 0
    END) AS 'Pass',
    SUM(CASE
        WHEN test_score <= 70
        THEN 1
        ELSE 0
    END) AS 'Fail'
FROM 
    test_scores
```

--

```SQL
SELECT 
    name, 
    score,
    CASE
        WHEN score > 70
            THEN 'Pass'
        WHEN score <= 70 AND score >= 60
            THEN 'Possible retake'
        WHEN score <60 
            THEN 'Fail'
    END outcome
FROM 
    test_scores
```

## 11b. Coalesce

```SQL
SELECT 
    first_name,
    last_name,
    COALESCE(phone,'N/A')
FROM 
    employees
```

--

```SQL
SELECT
    staff_id,
    COALESCE(
        hourly_rate*22*8, 
        weekly_rate*4, 
        monthly_rate
    ) monthly_salary
FROM
    salaries
```

--

```SQL
COALESCE(e1,e2,e3)
......
CASE
    WHEN e1 IS NOT NULL THEN e1
    WHEN e2 IS NOT NULL THEN e2
    ELSE e3
END
```

# 11c. Null If

```SQL
SELECT 
    name,
    email
FROM 
    clients
WHERE
    NULLIF(email, '') IS NULL

```

--

```SQL
SELECT 
    NULLIF(a,b)
........ equal to .....
CASE
    WHEN a=b
    THEN NULL
END
```